In [1]:
import pandas as pd
import json
from cleaning import domain, filter_vvo_tags
from files import list_to_file, file_to_list

In [2]:
def print_stats(total, sub, description):
    print(f"{description} {round(100*sub/total, 2)}% or {sub}")
    
# tag series will be something like df["tags"]
# returns count of entries that where classified as vegan, vegetarian, meat as well as the count and the tag_lists 
# that could not be categorized  
def categorization_quota(omniv_set, veg_set, vgn_set, tags_series):
    
    vegan = vegetarian = meat = vegan_and_vegetarian = empty = 0
    total = len(tags_series)
    not_classified = []
    
    for tag_list in tags_series:
        if tag_list == []:
            empty += 1
        
        tag_set = set(tag_list)
        if bool(omniv_set & tag_set):
            meat += 1
            continue
            
        if bool(vgn_veg_set & tag_set):
            vegan_and_vegetarian += 1
            continue
            
        if bool(veg_set & tag_set):
            vegetarian += 1
            continue

        if bool(vgn_set & tag_set):
            vegan += 1
            continue
            
        not_classified.append(tag_list)
    print(f"Total: {total}")
    print_stats(total, vegan, "Vegan:")
    print_stats(total, vegetarian, "Vegetarian: ")
    print_stats(total, vegan_and_vegetarian, "Vegan & Veg.:")
    print_stats(total, meat, "Meat:")
    print_stats(total, empty, "Empty:")
    print_stats(total, total-(empty+vegetarian+vegan+meat+vegan_and_vegetarian), "Could not be classified: ")

    return vegan, vegetarian, meat, len(not_classified), not_classified



In [3]:
data = pd.read_csv("Data/german-canteens(filtered).csv", sep="@", infer_datetime_format=True, index_col=0)
# evaluates tag_list as list (from string)
data["tags"] = list(map(eval, data["tags"]))

In [5]:
data[data["tags"].map(lambda x: len(x) >= 2)]

,mensa_id,date,meal_id,meal_name,vvo_status,tags,student_price,employee_price,guest_price
8,1,2023-01-09,13471727,hackbällchen jäger art mit champignons und zwi...,2,"[rind, schwein]",3.1,4.8,6.2
9,1,2023-01-09,13471729,"süßkartoffelcurry mit blumenkohl, belugalinsen...",0,"[knoblauch, vegan]",2.5,4.2,5.6
25,1,2023-01-16,13760636,mini-schweinegrillhaxe mit rotweinsoße dazu ei...,2,"[enth�lt alkohol, rind, schwein]",3.4,5.1,6.5
27,1,2023-01-16,13760592,soja-geröstel mit steckrüben-karotten-wirsingc...,0,"[knoblauch, vegan]",2.8,4.5,5.9
30,1,2023-01-17,13763693,steckrüben-karotten-wirsingcurry mit couscous,0,"[knoblauch, vegan]",2.2,3.9,5.3
...,...,...,...,...,...,...,...,...,...
700905,1838,2024-02-29,18750008,"tortelloni spinat- quinoa mit cherrytomaten, r...",1,"[vegetarisch, weizen, vegan]",2.0,2.0,2.0
700906,1838,2024-03-01,18750009,milchreis (mi) beerenfruchtsoße,1,"[vegetarisch, milch und laktose]",2.1,2.1,2.1
700933,1840,2024-02-14,18539635,karottenflan mit reis & \nlauch-gemüse ∆,1,"[lactose, ei]",NaN,NaN,NaN
700934,1840,2024-02-15,18539636,linsentaler auf apfel-\nsellerie-salat & senfg...,-1,"[senf, sellerie]",NaN,NaN,NaN


In [139]:
vgn_set = set(file_to_list("Data/tags/vgn_tags.txt"))
veg_set = set(file_to_list("Data/tags/veg_tags.txt"))
meat_set = set(file_to_list("Data/tags/meat_tags.txt"))
vgn_veg_set  = set(file_to_list("Data/tags/vgn_veg_tags.txt"))

#list_to_file("Data/tags/vgn_tags.txt", vgn_set)
#list_to_file("Data/tags/veg_tags.txt", veg_set)
#list_to_file("Data/tags/vgn_veg_tags.txt", vgn_set)
#list_to_file("Data/tags/meat_tags.txt", meat_set)

In [131]:
entry_count = len(data["tags"])
vegan = vegetarian = 0

#vgn_set = set(["vegan", "veganes gericht", "vegane speisen", "vegane speisen (v*)"])
#veg_set = set(["vegy", "vegi",  "vegetarisch", "vegetarian", "vegetarische speisen (V)","ohne fleisch", "vegetarische speisen (v)", "vegetarisches gericht", "vegetarische speisen"])
#omniv_set = set(['gemischtes fleisch','krebstiere','weichtiere','mit schweinefleisch','rind','geflügel','schweinefleisch','schwein','schweinfleisch','fische und daraus gewonnene erzeugnissse','gemischtes fleisch','fisch/fische','rind/kalb','enthält schweinefleisch','mit fleisch','mit geflügelfleisch','rindfleisch (r)','schweinefleisch (s)','geflügel (g)','fisch'])




all_canteens = []
hq_canteens = []

for index, meal in data.iterrows():
    all_canteens += [meal["mensa_id"]]
    
    tag_list = meal["tags"]
    tag_set = set(tag_list)
    
    if bool(omniv_set & tag_set):
        hq_canteens += [meal["mensa_id"]]
        continue
            
    if bool(vgn_veg_set & tag_set):
        hq_canteens += [meal["mensa_id"]]
        continue
    
    if bool(veg_set & tag_set):
        hq_canteens += [meal["mensa_id"]]
        continue

    if bool(vgn_set & tag_set):
        hq_canteens += [meal["mensa_id"]]
        continue


#print(f"Total: {entry_count}")
#print(f"Enthält tag \"vegan\": {100*round(vegan/entry_count, 4)}% or {vegan}")
#print(f"Enthält tag \"vegetarisch\": {100*round(vegetarian/entry_count, 4)}% or {vegetarian}")

In [132]:
all_canteens = list(set(all_canteens))
hq_canteens = list(set(hq_canteens))
len(hq_canteens)/len(all_canteens)

0.9124767225325885

In [100]:
hq_canteens

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 55,
 57,
 58,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 72,
 74,
 75,
 76,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 91,
 94,
 95,
 96,
 97,
 98,
 100,
 102,
 103,
 105,
 106,
 109,
 110,
 111,
 113,
 114,
 115,
 116,
 147,
 148,
 149,
 150,
 151,
 157,
 187,
 194,
 195,
 196,
 197,
 198,
 200,
 202,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 243,
 244,
 245,
 259,
 278,
 283,
 285,
 286,
 287,
 289,
 296,
 297,
 303,
 305,
 306,
 310,
 313,
 314,
 315,
 316,
 319,
 339,
 340,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 361,
 381,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 393,
 394,
 395,
 399,
 830,
 833,
 834,
 835,
 836,
 837,
 838,
 839,
 840,
 842,
 843,
 844,
 846,
 847,
 865,
 866,
 867,
 868,
 881,
 943,
 1091,
 1153,
 1154,
 1155,
 1156,
 1157,
 1158,
 1168,
 1169,
 1170,


In [133]:
hq_data = data[data["mensa_id"].isin(hq_canteens)]
print(len(hq_data)/len(data))
hq_data


0.9282421222140423


,mensa_id,date,meal_id,meal_name,vvo_status,tags,student_price,employee_price,guest_price
0,1,2023-01-04,13395373,Hähnchenschnitzel mit Waldpilzsoße dazu eine Gemüsebeilage und eine Sättigungsbeilage zur Auswahl,-1,[geflügel],2.8,4.5,5.9
1,1,2023-01-04,13395374,5 Grünkohl-Hanf-Bällchen mit Orangen-Ingwersoße dazu eine Gemüsebeilage und eine Sättigungsbeilage zur Auswahl,-1,[vegan],3.1,4.8,6.2
2,1,2023-01-04,13395375,Fish and Chips mit Mixed Pickles-Remoulade und marinierten Blattsalaten,-1,[fisch],3.4,5.1,6.5
3,1,2023-01-04,13395377,Der heutige Lieblingskuchen vom Mensa-Team: Fruchtiger Aprikosen-Quark-Kuchen vom Blech,-1,[vegetarisch],1.5,1.8,2.1
4,1,2023-01-05,13395378,Schweinesteak Appenzeller Art mit Röstzwiebeln und Reibekäse überbacken dazu Rahmsoße sowie eine Gemüsebeilage und eine Sättigungsbeilage zur Auswahl,-1,[schwein],3.4,5.1,6.5
...,...,...,...,...,...,...,...,...,...
688653,1840,2024-02-15,18539636,Linsentaler auf Apfel-\nSellerie-Salat & Senfglace \n*∆,-1,"[senf, sellerie]",NaN,NaN,NaN
688654,1840,2024-02-16,18539637,"Pasta mit Petersilienpesto, \ngeröstetem Kürbis und \nkaramellisierten Kernen *",-1,"[gluten, kerne]",NaN,NaN,NaN
688655,1840,2024-02-21,18633505,Rösti auf\nKräuter-Pilz-Gemüse &\nKnoblauchdip 1) 2),-1,[],NaN,NaN,NaN
688656,1840,2024-02-22,18633506,Couscous & geschmorter\nSpitzkohl mit\nPastinaken-Hummus 1),-1,[gluten],NaN,NaN,NaN


In [134]:
hq_domain = domain(hq_data["tags"])

In [135]:
list_to_file("Data/tags/hq_domain.txt", hq_domain)

In [138]:
vgn_veg_set

{'menü ist vegan',
 'vegan',
 'vegane speisen',
 'vegane speisen (v*)',
 'veganes  menü',
 'veganes energiespar-menü',
 'veganes gericht',
 'veganes menü',
 'veganes menü bei essen 2',
 'veganes menü im freeflow',
 'veganes one pot menü',
 'veganuary',
 'veganuary: veganes menü im freeflow'}

In [140]:
_,_,_, nc_count, nc_tag_lists = categorization_quota(vgn_set, veg_set, meat_set, data["tags"])

Total: 688677
Vegan: 5.86% or 40387
Vegetarian:  51.08% or 351769
Vegan & Veg.: 1.18% or 8104
Meat: 15.59% or 107362
Empty: 14.88% or 102451
Could not be classified:  11.41% or 78604


In [122]:
non_vv_domain = domain(nc_tag_lists)

In [123]:
list_to_file("Data/tags/non_vv_domain.txt", non_vv_domain)
print(len(non_vv_domain))
non_vv_domain

4745


['enth�lt alkohol',
 'knoblauch',
 'mit alkohol',
 'artger',
 'geschwefelt',
 'natürlich frisch',
 'enthält schwefeldioxid und sulfite',
 'enthält sesamsamen und sesamsamenerzeugnisse',
 'enthält sesamsamen',
 'kakaohaltige fettglasur',
 'klimateller',
 'enthält soja und sojaerzeugnisse',
 'regional',
 'outdoor',
 'glutenhaltiges getreide',
 'farbstoff',
 'weizen',
 'sellerie',
 'soja',
 'hafer',
 'konservierungsstoff',
 'sesam',
 'knackiger gartensalat',
 'senf',
 'dazu knoblauchmayonnaise und steakhouse pommes',
 'sulfit/ schwefeldioxid',
 'antioxidationsmittel',
 'dinkel',
 'gemüseschnitzel, paniert',
 'reis',
 'sesam karotten knuspersticks',
 'süßungsmitteln',
 'geschwärzt',
 'oder falafel',
 'mit falafel',
 'oder marinierten putengyros',
 'tomaten chili dip',
 'putengyros',
 'mit sojageschnetzeltem',
 'dazu china mie nudeln',
 'gerste',
 'dazu knoblauch-roggenbaguette',
 'lupine',
 'roggen',
 'an mariniertem feldsalat mit balsamico-dressing',
 'vegane polenta pizza mit buntem gemü

In [124]:
list_to_file("Data/tags/tag_domain.txt", domain(data["tags"]))